Логика работы такая. Одним из этапов регистрации/входа прохождение идентификации через камеру. Должна быть страница на которой человек должен дать доступ камере через которую он будет проходить аутентификацию. Он должен будет поместить лицо в два квадрата второй чуть меньше. Как только он включил камеру и до того момента пока он не поместит лицо в последний овал должны будут сохранятся кадры в jpg каждые 1-5 секунд. 

Все кадры нужно сохранить в базу отдельной сессией и название фоток должны содержать номер сессии.
Например сессия1-1 сессия1-2. К каждому аккаунту должен будет привязан номер сессии при первом прохождении. Но при этом сессии должны будут сохранятся при каждом прохождении.

Теперь после того как все кадры будут собраны нужно будет провести все фотографии через модель detector чтобы получить координаты лица, носа кончиков рта и глаз. После надо будет провести все кадры через bestframer чтобы получить score для каждого кадра. После нужно будет сохранить в базу в ту же сессию лучше 5 фотографии(если не будут проходить через трешхолд то может быть и меньше) еще одну папку в той же сессии. На этом заканчивается регистрация. Если же это не первый вход то нужно будет сравнить наилучшие кадры первой сессии и нынешней. Если всё ок то пропускаем дальше.

In [11]:
import os
import time
import cv2

from diploma.models.detector import FaceDetector
from diploma.drawing import draw_landmarks, draw_rectangles

detector = FaceDetector("E:\kbtu_courses\diploma_project\src\diploma\models\detector\weights\model_detector.onnx")

axes = (100, 150)    
angle = 0            
startAngle = 0       
endAngle = 360       
color = (0, 255, 0)  
thickness = 2  

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

output_dir = 'E:/kbtu_courses/diploma_project/src/diploma/pipeline/frames'

save_interval = 4
start_time = time.time()

end_capture = False

while True:
    ret, frame = cap.read()

    if time.time() - start_time >= save_interval:
        #filename = os.path.join(output_dir, f"frame_{int(time.time())}.jpg")
        #cv2.imwrite(filename, frame)
        start_time = time.time()

    boxes, landmarks = detector.detect(frame, input_size=(640, 640))

    center = (frame.shape[1] // 2, frame.shape[0] // 2)

    cv2.ellipse(frame, center, axes, angle, startAngle, endAngle, color, thickness)


    x1 = center[0] - axes[0] - 20
    y1 = center[1] - axes[1] - 20

    x2 = x1 + 2 * axes[0] + 40
    y2 = y1 + 2 * axes[1] + 40

    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

    frame = draw_rectangles(frame, bboxes = boxes, override=True)
    frame = draw_landmarks(frame, landmarks=landmarks)

    if (boxes[0][0] >= x1) and (boxes[0][1] >= y1) and (boxes[0][2] <= x2) and (boxes[0][3] <= y2):
        end_capture = True

    if not ret:
        print("Error: Could not capture frame.")
        break

    cv2.imshow('Camera', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


In [ ]:
face1 = cv2.imread("photo")
face2 = cv2.imread("photo")

face_bbox1, face_landmarks1 = FaceDetector(face1)
face_bbox2, face_landmarks2 = FaceDetector(face2)


aligned1 = align_face(face1, face_bbox1[0][:-1], face_landmarks1[0])
aligned2 = align_face(face2, face_bbox2[0][:-1], face_landmarks2[0])

embedding1 = FaceRecognition(aligned1)
embedding2 = FaceRecognition(aligned2)

sim = cosine_simularity(embedding1, embedding2)

print(sim)